a.	Сгенерите датафрейм размера 4х10 и наполните его случайными целыми числами в промежутке [1,10], выведите его на экран 

b.	Проведите преобразование: Для тех строк, в которых число во 2 колонке больше 3, поделите число в 4 колонке на число из 2 колонки

c.	Выведите получившийся датафрейм на экран, посчитайте разницу сумм колонок между исходным датафреймом и тем, что получился после преобразований

In [1]:
import numpy as np
import pandas as pd

In [2]:
df = pd.DataFrame(np.random.randint(1, 10, size=(10, 4)))

In [3]:
df1 = df.copy()
df1

,0,1,2,3
0,1,1,6,6
1,9,8,4,8
2,1,5,7,3
3,5,5,8,6
4,2,1,4,7
5,7,9,9,4
6,3,2,5,9
7,3,1,6,4
8,8,4,3,7
9,4,4,3,9


In [4]:
mask = df1[1] > 3

In [5]:
df1.loc[mask, 3] = (df1.loc[mask, 3] / df1.loc[mask, 1]).astype(int)

In [6]:
print(df, df1, sep='\n\n')

   0  1  2  3
0  1  1  6  6
1  9  8  4  8
2  1  5  7  3
3  5  5  8  6
4  2  1  4  7
5  7  9  9  4
6  3  2  5  9
7  3  1  6  4
8  8  4  3  7
9  4  4  3  9

   0  1  2  3
0  1  1  6  6
1  9  8  4  1
2  1  5  7  0
3  5  5  8  1
4  2  1  4  7
5  7  9  9  0
6  3  2  5  9
7  3  1  6  4
8  8  4  3  1
9  4  4  3  2


In [7]:
df.sum() - df1.sum()

0     0
1     0
2     0
3    32
dtype: int64

Откройте архивированный csv файл с данными. Там находятся логи о посещениях страниц сайта. Страницы находятся в поле ‘event’, событие event_9 считается целевым (покупка). Событие ‘lost’ означает уход пользователя с сайта. Вам нужно:

ii.	Вычислите средний промежуток времени между событиями внутри сессии (в микросекундах)


In [8]:
data = pd.read_csv(r'./test_task.csv.gz', parse_dates=['timestamp'])

In [9]:
data['next_time'] = data.groupby('session').timestamp.shift(-1)
data.head()

,client_id,session,event,timestamp,next_time
0,136651,1366511,sleep1->event_1,2020-05-01 00:00:00.000000000,2020-05-01 00:00:00.199838279
1,136651,1366511,sleep1->event_2,2020-05-01 00:00:00.199838279,2020-05-01 00:00:00.729752253
2,136651,1366511,sleep1->event_3,2020-05-01 00:00:00.729752253,2020-05-01 00:00:01.263877058
3,136651,1366511,sleep1->event_6,2020-05-01 00:00:01.263877058,2020-05-01 00:00:01.804570372
4,136651,1366511,lost,2020-05-01 00:00:01.804570372,NaT


In [10]:
(data.next_time - data.timestamp).mean()

Timedelta('0 days 00:00:00.431372')